In [177]:
import pandas as pd
import openpyxl
import os
import re
import numpy as np

### read file

In [3]:
excelFolder = "ITIS_RtoNR"
pandasList = []
for file in os.listdir(excelFolder):
    df = pd.read_excel(excelFolder + '/' + file)
    pandasList.append(df)

dfRtoNR = pd.concat(pandasList).copy()

#filter completed transaction
dfRtoNR_completed = dfRtoNR.query("`Combined Transaction Details.Transaction Status - Statement Status` == 'C'").copy()

In [2]:
#read id list by sector provided by labuan
excelFile = "data/raw/Data for ITIS - LFSA January 2021.xlsx"
wb_obj = openpyxl.load_workbook(excelFile) 

sheetNameList = wb_obj.sheetnames
entitiesDict = {}
for sector in sheetNameList:
    df = pd.read_excel(excelFile, sheet_name = sector)
    entitiesDict[sector] = list(df['Column1'].values)

#remove special chars for partnerships, trusts
entitiesDict2 = {i:[re.sub(r'/|-|\(|\)', '',x) for x in j] for i,j in entitiesDict.items()}

idListLabuan = [k for i,j in entitiesDict2.items() for k in j]

#read id list and licence no provided by labuan - 1st time
excelFile = "data/raw/Data  - STR, CTRs & ITIS  - LIBFC RA 2020.xlsx"
dfIDYL = pd.read_excel(excelFile)

idListYL = [re.sub(r'/|-|\(|\)', '',i) for i in dfIDYL['LL Number'].values]

licenseListYL = [re.sub(r'/|-|\(|\)', '',str(i)) for i in dfIDYL['LicenceNo'].values]

In [15]:
df['Combined Transaction Details.Transaction Status - Statement Status'].unique()

array(['I', 'C', nan, 'P'], dtype=object)

In [23]:
sum(pd.isnull(dfItis_completed['Combined Transaction Details.Transaction Date']))

58322

In [154]:
dfItis_completed['Year'] = pd.to_datetime(dfItis_completed['Combined Transaction Details.Transaction Date']).dt.year

In [157]:
dfItis_completed2020 = dfItis_completed.query("Year == 2020.0").copy()

### from yan ling aggregate data

In [5]:
#yan ling's aggregate table by entities
excelFile = "data/raw/Labuan_result_rtonr.xlsx"
dfYL = pd.read_excel(excelFile)

In [19]:
idListYLdf = dfYL[pd.notnull(dfYL['TOTAL_MYR'])]['ID_NO'].values

In [6]:
#get id provided by labuan fsa - version2
excelFile = "data/raw/Data for ITIS - LFSA January 2021.xlsx"
wb_obj = openpyxl.load_workbook(excelFile) 

sheetNameList = wb_obj.sheetnames
entitiesDict = {}
for sector in sheetNameList:
    df = pd.read_excel(excelFile, sheet_name = sector)
    entitiesDict[sector] = list(df['Column1'].values)

#remove slash symbol for partnerships, trusts
#sector: list of id
sectorIdDict = {i:[re.sub(r'/|-|\(|\)', '',x) for x in j] for i,j in entitiesDict.items()}

idListLabuan = [k for i,j in sectorIdDict.items() for k in j]

#get id provided by labuan fsa - version1
excelFile = "data/raw/Data  - STR, CTRs & ITIS  - LIBFC RA 2020.xlsx"
dfIDYL = pd.read_excel(excelFile)

idListYL = [re.sub('\/|-|\(|\)', '',i) for i in dfIDYL['LL Number'].values]

#LL Number: LicenceNo
dfIDYL['cleanLicenceNo'] = [re.sub('\/|-|\(|\)', '',str(i)) for i in dfIDYL['LicenceNo'].values]
#one LL Number may have more than one licenceNo
licenceDict = dfIDYL[pd.notnull(dfIDYL['cleanLicenceNo'])]\
              [['cleanLicenceNo', 'LL Number']].set_index('cleanLicenceNo').to_dict()['LL Number']

#list of licenceNo
licenseListYL = dfIDYL['cleanLicenceNo'].values

#licence no./id: sector
idLicenceToSectorDict = {k:i for i,j in sectorIdDict.items() for k in j}
licenceSectorDict = {i:idLicenceToSectorDict[j] for i,j in licenceDict.items()}
idLicenceToSectorDict.update(licenceSectorDict)

#get licence no./id list from idLicenceToSectorDict
idFinalList = idLicenceToSectorDict.keys()

In [142]:
[i for i in set(idFinalList).difference(idListYL) if i[0] in ['L']]

['LT0069',
 'LT0035',
 'LITC110003',
 'LT0044',
 'LMT0077',
 'LT0041',
 'LITC170058',
 'LITC170059',
 'LT0050',
 'LITC170050',
 'LITC110004',
 'LITC200065',
 'LITC150033',
 'LMT0080',
 'LT0043',
 'LT0056',
 'LMT0058',
 'LT0038',
 'LITC160041',
 'LT0057',
 'LITC150035',
 'LITC170056',
 'LITC140021',
 'LT0027',
 'LMT0084',
 'LMT0075',
 'LITC170053',
 'LITC110001',
 'LMT0081',
 'LMT0068',
 'LITC170054',
 'LMT0082',
 'LT0039',
 'LMT0087',
 'LT0034',
 'LT0065',
 'LT0028',
 'LT0063',
 'LT0032',
 'LT0042',
 'LITC130013',
 'LITC130018',
 'LT0026',
 'LITC150036',
 'LMT0070',
 'LITC130017',
 'LITC160044',
 'LMT0083',
 'LFAMB04',
 'LT0072',
 'LITC130016',
 'LMT0059 ',
 'LMT0065',
 'LITC200068',
 'LITC140027',
 'LITC160043',
 'LITC150030',
 'LMT0069',
 'LMT0088',
 'LT0074',
 'LFAMB03',
 'LMT0064',
 'LITC180061',
 'LITC120008',
 'LITC150031',
 'LFAMB08',
 'LITC140025',
 'LMT0047',
 'LT0030',
 'LT0078',
 'LT0048',
 'LITC160046',
 'LT0054',
 'LT0086',
 'LITC160039',
 'LITC110005',
 'LITC180060',
 'LI

In [23]:
#validation
#make sure every id/licence no. - version1 is contained in idLicenceToSectorDict
len(idListYL), sum([True for i in idListYL if i in idLicenceToSectorDict.keys()]), sum([True for i in idListYL if i in idFinalList])

(6210, 6210, 6210)

In [24]:
#make sure every id/licence no. from yan ling's aggregate table is contained in idLicenceToSectorDict
len(idListYLdf),  sum([True for i in idListYLdf if i in idLicenceToSectorDict.keys()]), sum([True for i in idListYLdf if i in idFinalList])

(7480, 7480, 7480)

In [327]:
#compare 2017
dfTemp2.shape[0], dfYL.query("YEAR == 2017")['TXN_COUNT'].sum()

(32725, 32705.0)

In [323]:
'LL04478' in idFinallist, 'LL04288' in idFinallist

(True, True)

In [324]:
idLicenceToSectorDict['LL04478'], idLicenceToSectorDict['LL04288']

('Leasing ', 'Leasing ')

In [322]:
'LL04478' in dfYL['ID_NO'].values, 'LL04288' in dfYL['ID_NO'].values

(False, False)

In [336]:
dfYL.shape

(11739, 6)

In [338]:
len(dfYL[pd.notnull(dfYL['YEAR'])]['ID_NO'].unique()), dfTemp2['']

2389

In [313]:
a = dfYL.query("YEAR == 2017")['ID_NO'].unique()

In [314]:
dfTemp2[(~(dfTemp2['BeneficiaryID'].isin(a))) &
                     (~(dfTemp2['SenderID'].isin(a)))]

,SenderName,SenderID,SenderCountry,SenderAccount,SenderRI,BeneficiaryName,BeneficiaryID,BeneficiaryCountry,BeneficiaryAccount,BeneficiaryRI,Purpose,Date,MYR,Year
7761908,PERISAI PACIFIC 101 L INC,LL04478,Malaysia,NaN,OCBC BANK (MALAYSIA) BERHAD,PERISAI OFFSHORE SDN BHD,NaN,LABUAN OFFSHORE,NaN,NaN,Unknown,2017-08-16,4000850.91,2017.0
7761909,PERISAI PACIFIC 101 L INC,LL04478,Malaysia,NaN,OCBC BANK (MALAYSIA) BERHAD,PERISAI PACIFIC 101 L INC,NaN,LABUAN OFFSHORE,NaN,NaN,Unknown,2017-09-19,3844833.11,2017.0
7815224,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,SINGAPORE TELECOMMUNICATIONS,NaN,Singapore,NaN,NaN,OTHER BUSINESS SEVICES,2017-08-17,126146.88,2017.0
7815423,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,CHESTERS TECHNOSERVICE PTE LTD,NaN,Singapore,NaN,NaN,GOODS,2017-09-29,81430.30,2017.0
7815424,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,CHESTER TECHNOSERVICE PTE LTD,NaN,Singapore,NaN,NaN,GOODS,2017-08-15,54748.05,2017.0
7815425,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,PACIFIC ENGINEERING AND SERVICE,NaN,Singapore,NaN,NaN,GOODS,2017-07-06,20283.05,2017.0
7815426,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,PACIFIC ENGINEERING AND SERVICE,NaN,Singapore,NaN,NaN,GOODS,2017-07-31,17214.16,2017.0
7815427,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,RAILKO BEARINGS S PTE LTD,NaN,Singapore,NaN,NaN,GOODS,2017-09-25,29810.03,2017.0
7815428,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,RAILKO BEARINGS S PTE LTD,NaN,Singapore,NaN,NaN,GOODS,2017-09-27,34003.80,2017.0
7815429,ALAM MARITIM L INC,LL04288,Malaysia,564847104776,MALAYAN BANKING BERHAD,PUMPQUIP ENGINEERING PTE LTD,NaN,Singapore,NaN,NaN,GOODS,2017-07-06,11173.17,2017.0


In [35]:
dfYL.groupby(["YEAR", "TXN_TYPE"])['TOTAL_MYR'].sum().reset_index()

,YEAR,TXN_TYPE,TOTAL_MYR
0,2016.0,RECEIVE,2.220020e+08
1,2016.0,SEND,3.281338e+08
2,2017.0,RECEIVE,3.254583e+10
3,2017.0,SEND,3.307535e+10
4,2018.0,RECEIVE,7.779353e+10
5,2018.0,SEND,9.282243e+10
6,2019.0,RECEIVE,1.315197e+11
7,2019.0,SEND,1.155764e+11


In [37]:
dfYL.groupby(["YEAR", "TXN_TYPE"])['TXN_COUNT'].count().reset_index()

,YEAR,TXN_TYPE,TXN_COUNT
0,2016.0,RECEIVE,3
1,2016.0,SEND,5
2,2017.0,RECEIVE,894
3,2017.0,SEND,741
4,2018.0,RECEIVE,1371
5,2018.0,SEND,1136
6,2019.0,RECEIVE,1712
7,2019.0,SEND,1618


In [301]:
dfTemp2.shape, dfYL['TXN_COUNT'].sum()

((223188, 14), 223127.0)

In [302]:
dfTemp2['MYR'].sum(), dfYL['TOTAL_MYR'].sum()

(483986476107.28, 483883415939.47)

In [282]:
dfTemp2.columns

Index(['SenderName', 'SenderID', 'SenderCountry', 'SenderAccount', 'SenderRI',
       'BeneficiaryName', 'BeneficiaryID', 'BeneficiaryCountry',
       'BeneficiaryAccount', 'BeneficiaryRI', 'Purpose', 'Date', 'MYR',
       'Year'],
      dtype='object')

In [341]:
senderList = set(dfTemp2['SenderID']) & set(idFinallist)
beneList = set(dfTemp2['BeneficiaryID']) & set(idFinallist)
len(senderList.union(beneList) & set(dfYL[pd.notnull(dfYL['YEAR'])]['ID_NO']))

2389

In [39]:
dfYL.groupby(["YEAR", "TXN_TYPE"])['TXN_COUNT'].sum().reset_index()

,YEAR,TXN_TYPE,TXN_COUNT
0,2016.0,RECEIVE,22.0
1,2016.0,SEND,30.0
2,2017.0,RECEIVE,15118.0
3,2017.0,SEND,17587.0
4,2018.0,RECEIVE,34064.0
5,2018.0,SEND,41928.0
6,2019.0,RECEIVE,46107.0
7,2019.0,SEND,68271.0


In [105]:
dfYL.head()

,ID_NO,YEAR,TXN_TYPE,TXN_COUNT,TOTAL_MYR
0,140115C,2017.0,SEND,1.0,6.526731e+05
1,930012C,2017.0,SEND,1.0,2.891949e+04
2,930016C,2019.0,SEND,1.0,1.161400e+08
3,940026C,2019.0,RECEIVE,1.0,2.512854e+06
4,960051C,2019.0,SEND,1.0,8.976334e+06


In [242]:
#map entities into sector
dfYL['Sector'] = dfYL['ID_NO'].map(idLicenceToSectorDict)

#aggregation
dfYLAggAmount = dfYL.groupby(["YEAR", "TXN_TYPE", "Sector"])['TOTAL_MYR'].sum().reset_index()

dfYLAggNoTrans = dfYL.groupby(["YEAR", "TXN_TYPE", "Sector"])['TXN_COUNT'].sum().reset_index()

In [243]:
dfYL['Sector'].unique()

array(['Banks ', 'Foundations ', 'Insurance ', 'Labuan Companies', 'LITC',
       'Leasing ', 'Money Broker ', 'Trust Company ',
       'Securities Licensee', 'Partnerships', 'Trusts'], dtype=object)

In [153]:
excelFile = "data/interim/2021-02-02_ItisAggTill2019.xlsx"
with pd.ExcelWriter(excelFile) as writer:
    dfYLAggNoTrans.to_excel(writer, sheet_name = 'NoTrans')
    dfYLAggAmount.to_excel(writer, sheet_name = 'TotalAmount')

In [163]:
#2020 - r to nr flat file
idFinallist = list(idLicenceToSectorDict.keys())
dfItis_match2020 = dfItis_completed2020.query("`Combined Transaction Details.Beneficiary - Remitter Identification Number` in @idFinallist").copy()

In [172]:
dfTemp = dfItis_completed.query("`Combined Transaction Details.Beneficiary - Remitter Identification Number` in @idFinallist").copy()

In [191]:
dfItis_match2020.shape, dfItis_completed2020.shape, dfTemp.shape, dfTemp2.shape, dfTemp3.shape

((0, 15), (1100563, 15), (69, 15), (223195, 14), (43071, 14))

In [177]:
excelFile = "data/raw/itisuptoApr2020.csv"
dfItis2 = pd.read_csv(excelFile)

D:\Users\figohjs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,6,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [180]:
dfItis2['Year'] = pd.to_datetime(dfItis2['Date']).dt.year

In [183]:
dfItisNo2020 = dfItis2.query("Year != 2020").copy()

dfItis2020 = dfItis2.query("Year == 2020").copy()

In [188]:
dfItisNo2020.shape, dfItis2020.shape

((9501850, 14), (767066, 14))

In [339]:
dfItisNo2020 = dfItis2.query("Year in [2016, 2017, 2018, 2019]").copy()

dfTemp2 = dfItisNo2020[((dfItisNo2020['BeneficiaryID'].isin(idFinallist))) |
                     ((dfItisNo2020['SenderID'].isin(idFinallist)))].copy()

In [190]:
dfTemp3 = dfItis2020[(dfItis2020['BeneficiaryID'].isin(idFinallist)) |
                     (dfItis2020['SenderID'].isin(idFinallist))].copy()

### By Country

In [12]:
excelFile = "data/raw/itisuptoApr2020.csv"
dfItisYL = pd.read_csv(excelFile)

D:\Users\figohjs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,6,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
dfItisYL.columns

Index(['SenderName', 'SenderID', 'SenderCountry', 'SenderAccount', 'SenderRI',
       'BeneficiaryName', 'BeneficiaryID', 'BeneficiaryCountry',
       'BeneficiaryAccount', 'BeneficiaryRI', 'Purpose', 'Date', 'MYR'],
      dtype='object')

In [26]:
dfItisYL['Year'] = pd.to_datetime(dfItisYL['Date']).dt.year

dfItisNo2020 = dfItisYL.query("Year in [2016, 2017, 2018, 2019]").copy()

dfItisNo2020 = dfItisNo2020[((dfItisNo2020['BeneficiaryID'].isin(idFinalList))) |
                            ((dfItisNo2020['SenderID'].isin(idFinalList)))].copy()

In [35]:
dfItisNo2020['Year'].unique()

array([2016., 2017., 2018., 2019.])

In [87]:
tableList = []
for year in dfItisNo2020['Year'].unique():
    colList = ['SenderID', 'Year', 'SenderCountry', 'MYR']
    dfTemp = dfItisNo2020[(dfItisNo2020['SenderID'].isin(idFinalList)) & 
                          (pd.notnull(dfItisNo2020['SenderID'])) &
                          (dfItisNo2020['Year'] == year)][colList].copy()
    dfTemp['TXN_TYPE'] = 'SEND'
    dfTemp.rename(columns = {'SenderID':"ID_NO", 'Year':"YEAR", 'SenderCountry': 'COUNTRY'}, inplace = True)
    tableList.append(dfTemp)
    
    colList = ['BeneficiaryID', 'Year', 'BeneficiaryCountry', 'MYR']
    dfTemp = dfItisNo2020[(dfItisNo2020['BeneficiaryID'].isin(idFinalList)) & 
                          (pd.notnull(dfItisNo2020['BeneficiaryID'])) &
                          (dfItisNo2020['Year'] == year)][colList].copy()
    dfTemp['TXN_TYPE'] = 'RECEIPT'
    dfTemp.rename(columns = {'BeneficiaryID':"ID_NO", 'Year':"YEAR", 'BeneficiaryCountry': 'COUNTRY'}, inplace = True)
    tableList.append(dfTemp)

In [88]:
dfItisNo2020_v2 = pd.concat(tableList).copy()

#map entities into sector
dfItisNo2020_v2['SECTOR'] = dfItisNo2020_v2['ID_NO'].map(idLicenceToSectorDict)

In [89]:
a = set(dfYL[pd.notnull(dfYL['YEAR'])]['ID_NO'].values)
b = set(dfItisNo2020_v2['ID_NO'].values)

#b has 2 more elements than a, a is a proper subset of b
len(a), len(b), len(a&b)

(2389, 2391, 2389)

In [98]:
b.difference(a)

{'LL04288', 'LL04478'}

In [173]:
#validation
dfItisNo2020_v2.groupby(["YEAR", "TXN_TYPE"])['ID_NO'].count().reset_index()

,YEAR,TXN_TYPE,ID_NO
0,2016.0,RECEIPT,22
1,2016.0,SEND,30
2,2017.0,RECEIPT,15119
3,2017.0,SEND,17606
4,2018.0,RECEIPT,34064
5,2018.0,SEND,41950
6,2019.0,RECEIPT,46109
7,2019.0,SEND,68288


In [92]:
dfYL.groupby(["YEAR", "TXN_TYPE"])['TXN_COUNT'].sum().reset_index()

,YEAR,TXN_TYPE,TXN_COUNT
0,2016.0,RECEIVE,22.0
1,2016.0,SEND,30.0
2,2017.0,RECEIVE,15118.0
3,2017.0,SEND,17587.0
4,2018.0,RECEIVE,34064.0
5,2018.0,SEND,41928.0
6,2019.0,RECEIVE,46107.0
7,2019.0,SEND,68271.0


In [93]:
dfAggNoTrans.groupby(["YEAR", "TXN_TYPE"])['ID_NO'].sum().reset_index()

,YEAR,TXN_TYPE,ID_NO
0,2016.0,RECEIPT,22
1,2016.0,SEND,30
2,2017.0,RECEIPT,15119
3,2017.0,SEND,17606
4,2018.0,RECEIPT,34064
5,2018.0,SEND,41950
6,2019.0,RECEIPT,46109
7,2019.0,SEND,68288


In [59]:
dfItisNo2020_v2.groupby(["YEAR", "TXN_TYPE"])['MYR'].sum().reset_index()

,YEAR,TXN_TYPE,MYR
0,2016.0,RECEIPT,2.220020e+08
1,2016.0,SEND,3.281338e+08
2,2017.0,RECEIPT,3.254614e+10
3,2017.0,SEND,3.309558e+10
4,2018.0,RECEIPT,7.779353e+10
5,2018.0,SEND,9.287342e+10
6,2019.0,RECEIPT,1.315200e+11
7,2019.0,SEND,1.156077e+11


In [104]:
dfAggAmount.groupby(["YEAR", "TXN_TYPE"])['TOTAL_MYR'].sum().reset_index()

,YEAR,TXN_TYPE,TOTAL_MYR
0,2016.0,RECEIPT,2.220020e+08
1,2016.0,SEND,3.281338e+08
2,2017.0,RECEIPT,3.254614e+10
3,2017.0,SEND,3.309558e+10
4,2018.0,RECEIPT,7.779353e+10
5,2018.0,SEND,9.287342e+10
6,2019.0,RECEIPT,1.315200e+11
7,2019.0,SEND,1.156077e+11


In [60]:
dfYL.groupby(["YEAR", "TXN_TYPE"])['TOTAL_MYR'].sum().reset_index()

,YEAR,TXN_TYPE,TOTAL_MYR
0,2016.0,RECEIVE,2.220020e+08
1,2016.0,SEND,3.281338e+08
2,2017.0,RECEIVE,3.254583e+10
3,2017.0,SEND,3.307535e+10
4,2018.0,RECEIVE,7.779353e+10
5,2018.0,SEND,9.282243e+10
6,2019.0,RECEIVE,1.315197e+11
7,2019.0,SEND,1.155764e+11


In [227]:
#aggregation
dfAggAmount = dfItisNo2020_v2.groupby(["YEAR", "TXN_TYPE", "SECTOR"])['MYR'].sum().reset_index()
dfAggAmount.rename(columns = {'MYR':'TOTAL_MYR'}, inplace = True)

dfAggNoTrans = dfItisNo2020_v2.groupby(["YEAR", "TXN_TYPE", "SECTOR"])['ID_NO'].count().reset_index()
dfAggNoTrans.rename(columns = {'ID_NO':'TXN_COUNT'}, inplace = True)

In [228]:
dfAggAmount['COUNTRY'].unique()

KeyError: 'COUNTRY'

In [95]:
dfAggNoTrans['COUNTRY'].unique()

array(['Malaysia'], dtype=object)

In [229]:
excelFile = "data/interim/2021-02-04_ItisAggTill2019.xlsx"
with pd.ExcelWriter(excelFile) as writer:
    dfAggNoTrans.to_excel(writer, sheet_name = 'NoTrans')
    dfAggAmount.to_excel(writer, sheet_name = 'TotalAmount')

In [165]:
tableList2 = []
for year in dfItisNo2020['Year'].unique():
    colList = ['SenderID', 'Year', 'BeneficiaryCountry', 'MYR']
    dfTemp = dfItisNo2020[(dfItisNo2020['SenderID'].isin(idFinalList)) & 
                          (pd.notnull(dfItisNo2020['SenderID'])) &
                          (dfItisNo2020['Year'] == year)][colList].copy()
    dfTemp['TXN_TYPE'] = 'SEND'
    dfTemp.rename(columns = {'SenderID':"ID_NO", 'Year':"YEAR", 'BeneficiaryCountry': 'COUNTRY'}, inplace = True)
    tableList2.append(dfTemp)
    
    colList = ['BeneficiaryID', 'Year', 'SenderCountry', 'MYR']
    dfTemp = dfItisNo2020[(dfItisNo2020['BeneficiaryID'].isin(idFinalList)) & 
                          (pd.notnull(dfItisNo2020['BeneficiaryID'])) &
                          (dfItisNo2020['Year'] == year)][colList].copy()
    dfTemp['TXN_TYPE'] = 'RECEIPT'
    dfTemp.rename(columns = {'BeneficiaryID':"ID_NO", 'Year':"YEAR", 'SenderCountry': 'COUNTRY'}, inplace = True)
    tableList2.append(dfTemp)

In [166]:
dfItisNo2020_v3 = pd.concat(tableList2).copy()

#map entities into sector
dfItisNo2020_v3['SECTOR'] = dfItisNo2020_v3['ID_NO'].map(idLicenceToSectorDict)

In [156]:
dfItisNo2020_v3['COUNTRY'].unique()

array(['United Arab Emirates', 'Singapore', 'Hong Kong', 'India',
       'Ireland', 'United States', 'China', 'Germany', 'Thailand',
       'Viet Nam', 'LABUAN OFFSHORE', 'Cambodia', 'Indonesia',
       'Netherlands', 'United Kingdom', 'Australia', 'Austria', 'Italy',
       'Latvia', 'Oman', 'Russian Federation', 'Saudi Arabia',
       'Sri Lanka', 'Switzerland', 'Argentina', 'Bahrain', 'Belgium',
       'Brunei Darussalam', 'Canada', 'Chile', 'Cyprus', 'Czech Republic',
       'Denmark', 'Estonia', 'Finland', 'Hungary', 'Isle of Man', 'Japan',
       'Jersey', 'Kenya', 'Korea, Republic of', 'Kuwait', 'Mauritius',
       'Moldova, Republic of', 'New Zealand', 'Norway', 'Panama',
       'Papua New Guinea', 'Philippines', 'South Africa', 'Spain',
       'Sweden', 'Taiwan, Province of China', 'Turkey', 'Uruguay',
       'Colombia', 'Kyrgyzstan', 'Peru', 'Bangladesh',
       'Bosnia and Herzegovina', 'Brazil', 'Maldives', 'France',
       'Guernsey', 'Qatar', 'Luxembourg', 'Lithuania', 'N

In [168]:
#aggregation
dfAggAmount = dfItisNo2020_v3.groupby(["YEAR", "COUNTRY", "TXN_TYPE", "SECTOR"])['MYR'].sum().reset_index()
dfAggAmount.rename(columns = {'MYR':'TOTAL_MYR'}, inplace = True)

dfAggNoTrans = dfItisNo2020_v3.groupby(["YEAR", "COUNTRY", "TXN_TYPE", "SECTOR"])['ID_NO'].count().reset_index()
dfAggNoTrans.rename(columns = {'ID_NO':'TXN_COUNT'}, inplace = True)

In [169]:
excelFile = "data/interim/2021-02-04_ItisAggTill2019_v2.xlsx"
with pd.ExcelWriter(excelFile) as writer:
    dfAggNoTrans.to_excel(writer, index = False, sheet_name = 'NoTrans')
    dfAggAmount.to_excel(writer, index = False, sheet_name = 'TotalAmount')

### NR to NR

In [96]:
excelFolder = "ITIS_NR-and-NR/data"
pandasList = []
for file in os.listdir(excelFolder):
    df = pd.read_excel(excelFolder + '/' + file)
    pandasList.append(df)
    
dfNRtoNR = pd.concat(pandasList).copy()

In [100]:
dfNRtoNR.columns

Index(['NR - MYR Amount', 'Country.Country Description',
       'NR Customer Details.Customer Beneficiary Remitter Account Number',
       'NR Customer Details.Customer Beneficiary Remitter Identification Number',
       'NR Customer Details.Customer Beneficiary Remitter Name',
       'NR Transaction Details.Beneficiary Remitter Identification Number',
       'NR Transaction Details.Name of Beneficiary Remitter',
       'NR Transaction Details.Transaction Date',
       'NR Transaction Details.Transaction Status',
       'NR Transaction Details.Transaction Type', 'Reporting Entity.RE Name',
       'Reporting Entity.RE Name 2', 'Reporting Period.Reporting Quarter',
       'Reporting Period.Reporting Year',
       'Purpose Code.Purpose Code Description',
       'Reporting Period.Reporting Month'],
      dtype='object')

In [102]:
amountCol = 'NR - MYR Amount'
transTypeCol = 'NR Transaction Details.Transaction Type'
yearCol = 'Reporting Period.Reporting Year'
idCol1 = 'NR Customer Details.Customer Beneficiary Remitter Identification Number'
idCol2 = 'NR Transaction Details.Beneficiary Remitter Identification Number'
countryCol = 'Country.Country Description'

dfNRtoNR_2020 = dfNRtoNR[(dfNRtoNR[yearCol] == 2020)].copy()

In [201]:
dfNRtoNR_2020.shape

(860577, 16)

In [198]:
tableList = []

for i in ['Other Payments', 'Other Receipts']:
    #match idCol1
    colList = [idCol1, yearCol, transTypeCol, amountCol, countryCol]
    dfTemp = dfNRtoNR_2020[(dfNRtoNR_2020[transTypeCol] == i) &
                           (dfNRtoNR_2020[idCol1].astype(str).isin(idFinalList))][colList].copy()
    dfTemp.rename(columns = {idCol1:"ID_NO", yearCol:"YEAR", 
                             transTypeCol:"TXN_TYPE", amountCol:"TOTAL_MYR",
                            countryCol: "COUNTRY"}, inplace = True)
    tableList.append(dfTemp)

    #match idCol2
    colList = [idCol2, yearCol, transTypeCol, amountCol, countryCol]
    dfTemp = dfNRtoNR_2020[(dfNRtoNR_2020[transTypeCol] == i) &
                           (dfNRtoNR_2020[idCol2].astype(str).isin(idFinalList))][colList].copy()
    dfTemp.rename(columns = {idCol2:"ID_NO", yearCol:"YEAR", 
                         transTypeCol:"TXN_TYPE", amountCol:"TOTAL_MYR", countryCol: "COUNTRY"}, inplace = True)
    tableList.append(dfTemp)

In [199]:
dfNRtoNR_2020Match = pd.concat(tableList).copy()
dfNRtoNR_2020Match = dfNRtoNR_2020Match[pd.notnull(dfNRtoNR_2020Match['ID_NO'])]

In [200]:
dfNRtoNR_2020Match['TXN_TYPE'] = dfNRtoNR_2020Match['TXN_TYPE'].map({"Other Payments":"SEND", "Other Receipts":"RECEIPT"})

dfNRtoNR_2020Match['SECTOR'] = dfNRtoNR_2020Match['ID_NO'].map(idLicenceToSectorDict)

In [201]:
dfNRtoNR_2020Match['TXN_TYPE'].value_counts()

SEND       88540
RECEIPT    57033
Name: TXN_TYPE, dtype: int64

In [132]:
dfNRtoNR_2020Match['SECTOR'].unique()

array(['Labuan Companies', 'Banks ', 'LITC', 'Insurance ', 'Leasing ',
       'Money Broker ', 'Trust Company ', 'Securities Licensee',
       'Foundations ', 'Trusts', 'Partnerships'], dtype=object)

In [230]:
dfAggAmount2020 = dfNRtoNR_2020Match.groupby(['YEAR', 'TXN_TYPE', 'SECTOR'])['TOTAL_MYR'].sum().reset_index()

dfAggNoTrans2020 = dfNRtoNR_2020Match.groupby(['YEAR', 'TXN_TYPE', 'SECTOR'])['ID_NO'].count().reset_index()
dfAggNoTrans2020.rename(columns = {'ID_NO':'TXN_COUNT'}, inplace = True)

In [266]:
dfAggNoTrans2020

,YEAR,TXN_TYPE,Sector,TXN_COUNT
0,2020,RECEIVE,Banks,1580
1,2020,RECEIVE,Foundations,28
2,2020,RECEIVE,Insurance,8155
3,2020,RECEIVE,LITC,5384
4,2020,RECEIVE,Labuan Companies,35680
5,2020,RECEIVE,Leasing,419
6,2020,RECEIVE,Money Broker,477
7,2020,RECEIVE,Partnerships,331
8,2020,RECEIVE,Securities Licensee,164
9,2020,RECEIVE,Trust Company,4812


In [231]:
dfAggNoTransFinal = pd.concat([dfAggNoTrans, dfAggNoTrans2020], axis = 0).copy()
dfAggAmountFinal = pd.concat([dfAggAmount, dfAggAmount2020], axis = 0).copy()

In [232]:
excelFile = "data/interim/2021-02-04_ItisAggFinal.xlsx"
with pd.ExcelWriter(excelFile) as writer:
    dfAggNoTransFinal.to_excel(writer, index = False, sheet_name = 'NoTrans')
    dfAggAmountFinal.to_excel(writer, index = False, sheet_name = 'TotalAmount')

### by Country

In [204]:
dfAggAmount2020 = dfNRtoNR_2020Match.groupby(['YEAR', 'COUNTRY', 'TXN_TYPE', 'SECTOR'])['TOTAL_MYR'].sum().reset_index()
dfAggNoTrans2020 = dfNRtoNR_2020Match.groupby(['YEAR', 'COUNTRY', 'TXN_TYPE', 'SECTOR'])['ID_NO'].count().reset_index()
dfAggNoTrans2020.rename(columns = {'ID_NO':'TXN_COUNT'}, inplace = True)

In [141]:
dfNRtoNR_2020Match['COUNTRY'].unique()

array(['Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Cambodia', 'Canada',
       'Cape Verde', 'Cayman Islands', 'Chile', 'China', "Cote D'Ivoire",
       'Croatia', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark',
       'Dominican Republic', 'Ecuador', 'Egypt', 'Estonia', 'Ethiopia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany',
       'Ghana', 'Gibraltar', 'Greece', 'Grenada', 'Guernsey', 'Guinea',
       'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland',
       'Isle of Man', 'Italy', 'Japan', 'Jersey', 'Kazakhstan', 'Kenya',
       'Korea, Republic of', 'Kuwait', 'LABUAN OFFSHORE',
       "Lao People's Democratic Republic", 'Latvia', 'Lebanon',
       'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Malawi',
       'Maldives', 'Malta',

In [207]:
dfAggNoTransFinal['YEAR'].unique()

array([2016., 2017., 2018., 2019., 2020.])

In [208]:
dfAggAmountFinal['YEAR'].unique()

array([2016., 2017., 2018., 2019., 2020.])

In [218]:
#take sender country if receipt and bene country if send
dfAggNoTransFinal = pd.concat([dfAggNoTrans, dfAggNoTrans2020], axis = 0).copy()
dfAggAmountFinal = pd.concat([dfAggAmount, dfAggAmount2020], axis = 0).copy()

In [209]:
excelFile = "data/interim/2021-02-04_ItisAggFinal_v2.xlsx"
with pd.ExcelWriter(excelFile) as writer:
    dfAggNoTransFinal.to_excel(writer, index = False, sheet_name = 'NoTrans')
    dfAggAmountFinal.to_excel(writer, index = False, sheet_name = 'TotalAmount')

In [219]:
dfMerged = dfAggNoTransFinal.merge(dfAggAmountFinal, how = 'outer', 
                                    on = ['TXN_TYPE', 'YEAR', 'COUNTRY', 'SECTOR'])
dfMerged

,YEAR,COUNTRY,TXN_TYPE,SECTOR,TXN_COUNT,TOTAL_MYR
0,2016.0,China,SEND,LITC,1,13136585.00
1,2016.0,Hong Kong,RECEIPT,LITC,10,2753223.84
2,2016.0,Hong Kong,SEND,LITC,1,28371.39
3,2016.0,Jordan,SEND,LITC,2,23387258.70
4,2016.0,LABUAN OFFSHORE,SEND,Labuan Companies,6,4307169.26
...,...,...,...,...,...,...
3374,2020.0,Yemen,SEND,Labuan Companies,4,4491.42
3375,2020.0,Zambia,RECEIPT,LITC,1,382505.00
3376,2020.0,Zambia,RECEIPT,Labuan Companies,1,62941.55
3377,2020.0,Zambia,SEND,Labuan Companies,88,3095125.79


In [221]:
dfMergedSend = dfMerged.query("TXN_TYPE == 'SEND'")
dfMergedReceipt = dfMerged.query("TXN_TYPE == 'RECEIPT'")

In [222]:
dfCollapseSend = pd.pivot_table(dfMergedSend, values = ['TXN_COUNT', 'TOTAL_MYR'],
                              index = ['TXN_TYPE', 'COUNTRY', 'SECTOR'],
                          columns = ['YEAR'], aggfunc=np.sum, fill_value=0)
dfCollapseSend

TOTAL_MYR                        \
YEAR                                     2016.0    2017.0      2018.0   
TXN_TYPE COUNTRY     SECTOR                                             
SEND     Afghanistan Labuan Companies       0.0       0.0    41066.84   
         Albania     Labuan Companies       0.0       0.0  1158918.23   
         Algeria     Insurance              0.0       0.0   796017.46   
                     Labuan Companies       0.0       0.0        0.00   
                     Money Broker           0.0       0.0        0.00   
...                                         ...       ...         ...   
         Yemen       Labuan Companies       0.0       0.0        0.00   
                     Money Broker           0.0       0.0        0.00   
         Zambia      Insurance              0.0       0.0    35162.98   
                     Labuan Companies       0.0  213975.0   902339.77   
         Zimbabwe    Labuan Companies       0.0       0.0        0.00   

                                                              TXN_COUNT  \
YEAR                                       2019.0      2020.0    2016.0   
TXN_TYPE COUNTRY     SECTOR                                               
SEND     Afghanistan Labuan Companies    17205.33        0.00         0   
         Albania     Labuan Companies   126960.00     8613.31         0   
         Algeria     Insurance          240198.28    25794.75         0   
                     Labuan Companies    11923.80   141471.58         0   
                     Money Broker            0.00      389.40         0   
...                                           ...         ...       ...   
         Yemen       Labuan Companies    72127.75     4491.42         0   
                     Money Broker        26964.08        0.00         0   
         Zambia      Insurance          418284.25        0.00         0   
                     Labuan Companies  1320119.53  3095125.79         0   
         Zimbabwe    Labuan Companies        0.00    44848.24         0   

                                                                   
YEAR                                  2017.0 2018.0 2019.0 2020.0  
TXN_TYPE COUNTRY     SECTOR                                        
SEND     Afghanistan Labuan Companies      0      2      3      0  
         Albania     Labuan Companies      0      3      3      3  
         Algeria     Insurance             0      1      1      1  
                     Labuan Companies      0      0      2      3  
                     Money Broker          0      0      0      1  
...                                      ...    ...    ...    ...  
         Yemen       Labuan Companies      0      0      3      4  
                     Money Broker          0      0      3      0  
         Zambia      Insurance             0      2      7      0  
                     Labuan Companies      1     15     16     88  
         Zimbabwe    Labuan Companies      0      0      0      6  

[654 rows x 10 columns]

In [217]:
dfMergedReceipt['YEAR'].unique()

array([2016., 2017., 2018., 2019.])

In [223]:
dfCollapseReceipt = pd.pivot_table(dfMergedReceipt, values = ['TXN_COUNT', 'TOTAL_MYR'],
                              index = ['TXN_TYPE', 'COUNTRY', 'SECTOR'],
                          columns = ['YEAR'], aggfunc=np.sum, fill_value=0)
dfCollapseReceipt

TOTAL_MYR              \
YEAR                                                 2016.0      2017.0   
TXN_TYPE COUNTRY                 SECTOR                                   
RECEIPT  Afghanistan             Banks                  0.0        0.00   
                                 Insurance              0.0    16746.74   
                                 LITC                   0.0   342741.30   
                                 Labuan Companies       0.0        0.00   
                                 Trust Company          0.0        0.00   
...                                                     ...         ...   
         Virgin Islands, British Trust Company          0.0        0.00   
         Western Sahara          Labuan Companies       0.0        0.00   
         Yemen                   Labuan Companies       0.0  1069875.64   
         Zambia                  LITC                   0.0        0.00   
                                 Labuan Companies       0.0        0.00   

                                                                              \
YEAR                                                     2018.0       2019.0   
TXN_TYPE COUNTRY                 SECTOR                                        
RECEIPT  Afghanistan             Banks             1.390342e+08  36598741.77   
                                 Insurance         8.964820e+03         0.00   
                                 LITC              0.000000e+00         0.00   
                                 Labuan Companies  1.670078e+07    172150.56   
                                 Trust Company     1.763483e+04         0.00   
...                                                         ...          ...   
         Virgin Islands, British Trust Company     2.000000e+04         0.00   
         Western Sahara          Labuan Companies  0.000000e+00     21927.90   
         Yemen                   Labuan Companies  2.604744e+05   5786608.57   
         Zambia                  LITC              0.000000e+00         0.00   
                                 Labuan Companies  2.449408e+04      1541.92   

                                                              TXN_COUNT  \
YEAR                                                   2020.0    2016.0   
TXN_TYPE COUNTRY                 SECTOR                                   
RECEIPT  Afghanistan             Banks                   0.00         0   
                                 Insurance               0.00         0   
                                 LITC                    0.00         0   
                                 Labuan Companies  5959070.25         0   
                                 Trust Company        4051.20         0   
...                                                       ...       ...   
         Virgin Islands, British Trust Company       55556.28         0   
         Western Sahara          Labuan Companies        0.00         0   
         Yemen                   Labuan Companies        0.00         0   
         Zambia                  LITC               382505.00         0   
                                 Labuan Companies    62941.55         0   

                                                                               
YEAR                                              2017.0 2018.0 2019.0 2020.0  
TXN_TYPE COUNTRY                 SECTOR                                        
RECEIPT  Afghanistan             Banks                 0      2      1      0  
                                 Insurance             1      2      0      0  
                                 LITC                  2      0      0      0  
                                 Labuan Companies      0      6      2      8  
                                 Trust Company         0      1      0      2  
...                                                  ...    ...    ...    ...  
         Virgin Islands, British Trust Company         0      1      0      1  
         Western Sahara    

In [186]:
excelFile = "data/interim/2021-02-04_ItisAggFinal_collapse.xlsx"
dfCollapse.to_excel(excelFile)

In [226]:
excelFile = "data/interim/2021-02-04_ItisAggFinal_collapse2.xlsx"
with pd.ExcelWriter(excelFile) as writer:
    dfCollapseSend.to_excel(writer, sheet_name = 'Send')
    dfCollapseReceipt.to_excel(writer, sheet_name = 'Receipt')